### Установки и импорты

In [1]:
!pip install pymorphy3

In [2]:
!pip install corus

In [3]:
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-3yz8kc9q
  Running command git clone --filter=blob:none --quiet https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-3yz8kc9q
  Resolved https://github.com/lopuhin/python-adagram.git to commit cf3639f10d6a1efbcb602f45a1da89ef55ce5794
  Preparing metadata (setup.py) ... done


In [4]:
!pip install transformers

In [5]:
from collections import defaultdict
from typing import Iterable, Tuple, Callable, Generator, Union, Dict

import numpy as np
import pandas as pd
from tqdm import tqdm

from pymorphy3 import MorphAnalyzer
from nltk import tokenize
from corus import load_morphoru_rnc, load_ud_syntag
from corus.sources.ud import UDSent
from corus.sources.morphoru import MorphoSent

import adagram
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from dict_crawler import LexicographyCrawler
from wsi_pipeline import ContextEmb, WSIPipeline, BertEmbeddings

## 1 - Выберите 5 неоднозначных лексем и соберите для них словарные значения (толкования). Можно использовать МАС или любой другой толковый словарь. Обязательно укажите, каким словарем вы пользовались. - 1 балл за ручной сбор данных, 1 балл за краулинг



In [6]:
selected_words = [
    'клетка',
    'фигура',
    'карта',
    'анализ',
    'функция',
]

#### краулинг

(имплементация краулера в файле dict_crawler.py)

In [7]:
crawler = LexicographyCrawler()

In [ ]:
words_definitions = {}
for w in selected_words:
    defin = crawler.search_definitions(w)
    words_definitions[w] = defin

In [ ]:
## Для токенизации буду использовать wodpunct tokenize, а для лемматизации самый вероятный разбор pymorphy

morph = MorphAnalyzer()
tokenizer = tokenize.WordPunctTokenizer()
def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenizer.tokenize(text) if word]
    return words

In [ ]:
# сделаю из этого датафрейм

definitions = []
for w, sents in words_definitions.items():
    for i, sent in enumerate(sents, start=1):
        definitions.append((w, i, sent, ' '.join(normalize(sent))))

definitions_dataset = pd.DataFrame(definitions, columns=['word', 'sense_id', 'definition', 'sent_lemmatized'])

In [ ]:
# и сохраню, чтобы потом не приходилось перезапускать

definitions_dataset.to_csv('data/definitions_dataset.csv', sep='\t', header=True, index=True)

#### загрузка посчитанного раннее

(здесь в выводе могут быть столбцы, которые будут полученя дальше, потому что я по ходу обновляла эту таблицу и сохраняла новые результаты)

In [7]:
definitions_dataset = pd.read_csv('data/definitions_dataset.csv', sep='\t', header=0, index_col=0)
definitions_dataset.head()

,word,sense_id,definition,sent_lemmatized,adagram_sense,sent,word_idx,bert_sense
0,клетка,1,Помещение для птиц и животных со стенками из ж...,помещение для птица и животное с стенка из жел...,2,Клетка - это помещение для птиц и животных со ...,0,0
1,клетка,2,"разг. Маленькая комната, тесное помещение.","разг . маленький комната , тесный помещение .",2,"Клетка - это разг. маленькая комната, тесное ...",0,0
2,клетка,3,"Способ укладки каких-л. материалов (бревен, др...","способ укладка какой - л . материал ( бревно ,...",2,Клетка - это способ укладки каких-л. материало...,0,0
3,клетка,4,"Четырехугольник, изображенный на поверхности ч...","четырёхугольник , изобразить на поверхность чт...",2,"Клетка - это четырехугольник, изображенный на ...",0,0
4,клетка,5,"Элементарная живая система, основа строения и ...","элементарный живой система , основа строение и...",4,"Клетка - это элементарная живая система, основ...",0,0


## 2 - Выберите один или несколько корпусов коллекции Corus, извлеките оттуда все предложения, где содержатся ваши неоднозначные слова.

#### сбор данных

Возьму корпуса с морфологическое разметкой (один с morphoRuEval + 5 UD корпусов), чтобы не надо было самой лемматизировать тексты. И буду так же сохранять предложения обычные и лемматизированные (это потом пригодится)

In [ ]:
!wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/RNC_texts.rar
!unrar x RNC_texts.rar
!rm RNC_texts.rar

--2023-12-05 11:25:52--  https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/RNC_texts.rar
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dialogue-evaluation/morphoRuEval-2017/master/RNC_texts.rar [following]
--2023-12-05 11:25:52--  https://raw.githubusercontent.com/dialogue-evaluation/morphoRuEval-2017/master/RNC_texts.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5775644 (5.5M) [application/octet-stream]
Saving to: ‘RNC_texts.rar’

RNC_texts.rar       100%[===================>]   5.51M  --.-KB/s    in 0.1s    

2023-12-05 11:25:52 (54.4 MB/s) - ‘RNC_texts.rar’ saved [57

In [ ]:
!wget https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-dev.conllu
!wget https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-test.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu

--2023-12-05 11:43:25--  https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-dev.conllu
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu [following]
--2023-12-05 11:43:25--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14704579 (14M) [text/plain]
Saving to: ‘ru_syntagrus-ud-dev.conllu’

ru_syntagrus-ud-dev 100%[===================>]  14.02M  --.-KB/s    in 0.1s    

2023-1

In [ ]:
corpora_loads = [
    (load_ud_syntag, 'ru_syntagrus-ud-dev.conllu'),
    (load_ud_syntag, 'ru_syntagrus-ud-test.conllu'),
    (load_ud_syntag, 'ru_syntagrus-ud-train-a.conllu'),
    (load_ud_syntag, 'ru_syntagrus-ud-train-b.conllu'),
    (load_ud_syntag, 'ru_syntagrus-ud-train-c.conllu'),
    (load_morphoru_rnc, 'RNCgoldInUD_Morpho.conll')]

In [ ]:
def is_word_entry(morpho_sent, search_lemma) -> bool:
    for t in morpho_sent.tokens:
        if t.lemma == search_lemma:
            return True
    return False

In [ ]:
def search_in_corpus(word: str,
                     load_func: Callable[
                          [str],
                          Generator[Union[MorphoSent, UDSent], None, None]],
                     corpus_path: str) -> Iterable[Tuple[str, str]]:
    entries = []
    sent_generator = load_func(corpus_path)
    for sent in sent_generator:
        if is_word_entry(sent, word):
            sent_text = ' '.join([t.text for t in sent.tokens if t.text])
            sent_lemmas = ' '.join([t.lemma for t in sent.tokens if t.lemma])
            entries.append((sent_text, sent_lemmas))
    return entries

In [ ]:
selected_word_contexts = defaultdict(list)
for w in selected_words:
    for corpus in corpora_loads:
        contexts = search_in_corpus(w, *corpus)
        selected_word_contexts[w].extend(contexts)

dataset = [(w, *sent) for w, sents in selected_word_contexts.items() for sent in sents]
dataset = pd.DataFrame(dataset, columns=['word', 'sent', 'sent_lemmatized'])
dataset.head()

,word,sent,sent_lemmatized
0,клетка,"Разработчики , занятые в этой отрасли , отказа...","разработчик , занять в этот отрасль , отказать..."
1,клетка,"Кстати , клетка в целом и одноклеточные органи...","кстати , клетка в целое и одноклеточный органи..."
2,клетка,"Одно из наиболее известных приспособлений , по...","один из наиболее известный приспособление , за..."
3,клетка,А с развитием нанотехнологий ученые получают в...,а с развитие нанотехнология ученый получать во...
4,клетка,"Термин "" нанотехнологии "" был популяризован ещ...","термин "" нанотехнология "" быть популяризироват..."


In [ ]:
for w, sents in selected_word_contexts.items():
    print(w, len(sents))

клетка 179
фигура 236
карта 199
анализ 358
функция 409


In [ ]:
# еще сохраню позицию таргетного слова, это потом будет нужно для берта

def find_word_idx(row: pd.Series) -> int:
    word = row.word
    lemmas = row.sent_lemmatized.split()
    idx = lemmas.index(word)
    return idx

dataset['word_idx'] = dataset.apply(find_word_idx, axis=1)

In [ ]:
dataset.head()

,word,sent,sent_lemmatized,word_idx
0,клетка,"Разработчики , занятые в этой отрасли , отказа...","разработчик , занять в этот отрасль , отказать...",14
1,клетка,"Кстати , клетка в целом и одноклеточные органи...","кстати , клетка в целое и одноклеточный органи...",2
2,клетка,"Одно из наиболее известных приспособлений , по...","один из наиболее известный приспособление , за...",8
3,клетка,А с развитием нанотехнологий ученые получают в...,а с развитие нанотехнология ученый получать во...,37
4,клетка,"Термин "" нанотехнологии "" был популяризован ещ...","термин "" нанотехнология "" быть популяризироват...",38


In [ ]:
# так же сохраняю, чтобы не пересобирать корпус

dataset.to_csv('data/wsd_dataset.csv', sep='\t', header=True, index=True)

#### загрузка собранного корпуса

In [9]:
dataset = pd.read_csv('data/wsd_dataset.csv', sep='\t', header=0, index_col=0)
dataset.head()

,word,sent,sent_lemmatized,word_idx,adagram_sense,bert_sense
0,клетка,"Разработчики , занятые в этой отрасли , отказа...","разработчик , занять в этот отрасль , отказать...",14,4,0
1,клетка,"Кстати , клетка в целом и одноклеточные органи...","кстати , клетка в целое и одноклеточный органи...",2,2,0
2,клетка,"Одно из наиболее известных приспособлений , по...","один из наиболее известный приспособление , за...",8,2,1
3,клетка,А с развитием нанотехнологий ученые получают в...,а с развитие нанотехнология ученый получать во...,37,2,0
4,клетка,"Термин "" нанотехнологии "" был популяризован ещ...","термин "" нанотехнология "" быть популяризироват...",38,2,1


## 3 - Разбейте полученные контексты по значениям

Разбейте полученные контексты по значениям двумя любыми способами (вы можете использовать способы с семинара, заменить там эмбеддинги, реализовать какой-то свой способ). Напоминалка: в случае с AdaGram вы сразу получаете какой-то номер значения, с ELMo и другими контекстуальными эмбеддингами — нужно кластеризовать. - 2 балла

### AdaGram

In [ ]:
!curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > all.a010.p10.d300.w5.m100

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1394M  100 1394M    0     0  48.4M      0  0:00:28  0:00:28 --:--:-- 47.9M


In [ ]:
vm = adagram.VectorModel.load('/content/all.a010.p10.d300.w5.m100')

In [ ]:
def adagram_sense(row) -> int:
    means = vm.disambiguate(row.word, row.sent_lemmatized)
    i = np.argmax(means)
    return i

In [ ]:
dataset['adagram_sense'] = dataset.apply(adagram_sense, axis=1)
dataset.to_csv('data/wsd_dataset.csv', sep='\t', header=True, index=True)

/usr/local/lib/python3.10/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


In [ ]:
dataset = pd.read_csv('data/wsd_dataset.csv', sep='\t', header=0, index_col=0)
dataset.head()

,word,sent,sent_lemmatized,word_idx,adagram_sense
0,клетка,"Разработчики , занятые в этой отрасли , отказа...","разработчик , занять в этот отрасль , отказать...",14,4
1,клетка,"Кстати , клетка в целом и одноклеточные органи...","кстати , клетка в целое и одноклеточный органи...",2,2
2,клетка,"Одно из наиболее известных приспособлений , по...","один из наиболее известный приспособление , за...",8,2
3,клетка,А с развитием нанотехнологий ученые получают в...,а с развитие нанотехнология ученый получать во...,37,2
4,клетка,"Термин "" нанотехнологии "" был популяризован ещ...","термин "" нанотехнология "" быть популяризироват...",38,2


### BERT

код для берта в wsi_pipeline.py - там реализована векторизация слов и кластеризация контекстов

In [ ]:
bert_model = BertEmbeddings('bert-base-multilingual-cased')

In [ ]:
dataloader = DataLoader(ContextEmb(dataset), batch_size = 8, shuffle=False)
pipeline = WSIPipeline(dataset)

In [ ]:
# векторизуем с помощью берта
pipeline.vectorize_contexts(dataloader, bert_model)

# сохраняем, чтобы не пересчитывать
pipeline.save_embeddings('data/word_entries_embeddings.npy')

100%|██████████| 173/173 [08:00<00:00,  2.78s/it]


In [ ]:
# или грузим посчитанные раннее эмбеддинги
pipeline.load_embeddings('data/word_entries_embeddings.npy')

In [ ]:
# кластеризуем с помощью kmeans
predicted_labels = pipeline.cluster_all_dataset(k_range=range(2, 15), fit=True)

# сохраняем модели кластеризации для дальнейшего использования
pipeline.save_cluster_models('data/cluster_models.pkl')

In [ ]:
# или грузим уже предобученные модели кластеризации
pipeline.load_cluster_models('data/cluster_models.pkl')

# и снова получаем предсказания
predicted_labels = pipeline.cluster_all_dataset(k_range=range(2, 15), fit=False)

In [ ]:
# теперь добавим нужный столбец в наш датасет
for w, predicted in predicted_labels.items():
    dataset.loc[predicted[0], 'bert_sense'] = predicted[1]

dataset = dataset.astype({'bert_sense': 'int32'})
dataset.to_csv('data/wsd_dataset.csv', sep='\t', header=True, index=True)

In [ ]:
# загрузка уже посчитанных данных

dataset = pd.read_csv('data/wsd_dataset.csv', sep='\t', header=0, index_col=0)
dataset.head()

,word,sent,sent_lemmatized,word_idx,adagram_sense,bert_sense
0,клетка,"Разработчики , занятые в этой отрасли , отказа...","разработчик , занять в этот отрасль , отказать...",14,4,0
1,клетка,"Кстати , клетка в целом и одноклеточные органи...","кстати , клетка в целое и одноклеточный органи...",2,2,0
2,клетка,"Одно из наиболее известных приспособлений , по...","один из наиболее известный приспособление , за...",8,2,1
3,клетка,А с развитием нанотехнологий ученые получают в...,а с развитие нанотехнология ученый получать во...,37,2,0
4,клетка,"Термин "" нанотехнологии "" был популяризован ещ...","термин "" нанотехнология "" быть популяризироват...",38,2,1


## 4 - Возьмите словарные толкования для каждого значения и припишите им значение автоматически теми же способами, что и в п.2. - 1 балл

### AdaGram

In [ ]:
# выводим значения
definitions_dataset['adagram_sense'] = definitions_dataset.apply(adagram_sense, axis=1)
definitions_dataset.head()

/usr/local/lib/python3.10/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


,word,sense_id,definition,sent_lemmatized,adagram_sense
0,клетка,1,Помещение для птиц и животных со стенками из ж...,помещение для птица и животное с стенка из жел...,2
1,клетка,2,"разг. Маленькая комната, тесное помещение.","разг . маленький комната , тесный помещение .",2
2,клетка,3,"Способ укладки каких-л. материалов (бревен, др...","способ укладка какой - л . материал ( бревно ,...",2
3,клетка,4,"Четырехугольник, изображенный на поверхности ч...","четырёхугольник , изобразить на поверхность чт...",2
4,клетка,5,"Элементарная живая система, основа строения и ...","элементарный живой система , основа строение и...",4


In [ ]:
# чекпойнт
definitions_dataset.to_csv('data/definitions_dataset.csv', sep='\t', header=True, index=True)

In [ ]:
definitions_dataset = pd.read_csv('data/definitions_dataset.csv', sep='\t', header=0, index_col=0)
definitions_dataset.head()

### BERT

Чтобы воспользоваться тем же алгоритмом берта, я превращу определения в конструкцию типа "Слово - это definition" и сделаю word_idx = 0 (поскольку везде таргетное слово будет стоять первым)

In [ ]:
def full_definition(row: pd.Series) -> str:
    word = row.word.title()
    definition = row.definition.lower()
    return f'{word} - это {definition}'

In [ ]:
definitions_dataset['sent'] = definitions_dataset.apply(full_definition, axis=1)
definitions_dataset['word_idx'] = 0
definitions_dataset.head()

,word,sense_id,definition,sent_lemmatized,adagram_sense,sent,word_idx
0,клетка,1,Помещение для птиц и животных со стенками из ж...,помещение для птица и животное с стенка из жел...,2,Клетка - это помещение для птиц и животных со ...,0
1,клетка,2,"разг. Маленькая комната, тесное помещение.","разг . маленький комната , тесный помещение .",2,"Клетка - это разг. маленькая комната, тесное ...",0
2,клетка,3,"Способ укладки каких-л. материалов (бревен, др...","способ укладка какой - л . материал ( бревно ,...",2,Клетка - это способ укладки каких-л. материало...,0
3,клетка,4,"Четырехугольник, изображенный на поверхности ч...","четырёхугольник , изобразить на поверхность чт...",2,"Клетка - это четырехугольник, изображенный на ...",0
4,клетка,5,"Элементарная живая система, основа строения и ...","элементарный живой система , основа строение и...",4,"Клетка - это элементарная живая система, основ...",0


In [ ]:
# чекпойнт
definitions_dataset.to_csv('data/definitions_dataset.csv', sep='\t', header=True, index=True)

In [ ]:
definitions_dataset = pd.read_csv('data/definitions_dataset.csv', sep='\t', header=0, index_col=0)
definitions_dataset.head()

,word,sense_id,definition,sent_lemmatized,adagram_sense,sent,word_idx
0,клетка,1,Помещение для птиц и животных со стенками из ж...,помещение для птица и животное с стенка из жел...,2,Клетка - это помещение для птиц и животных со ...,0
1,клетка,2,"разг. Маленькая комната, тесное помещение.","разг . маленький комната , тесный помещение .",2,"Клетка - это разг. маленькая комната, тесное ...",0
2,клетка,3,"Способ укладки каких-л. материалов (бревен, др...","способ укладка какой - л . материал ( бревно ,...",2,Клетка - это способ укладки каких-л. материало...,0
3,клетка,4,"Четырехугольник, изображенный на поверхности ч...","четырёхугольник , изобразить на поверхность чт...",2,"Клетка - это четырехугольник, изображенный на ...",0
4,клетка,5,"Элементарная живая система, основа строения и ...","элементарный живой система , основа строение и...",4,"Клетка - это элементарная живая система, основ...",0


Для предсказания значений по определениям я буду использовать модели кластеризации, предобученные на контекстах слова (потому что нам нет смысла кластеризовать определения между собой, нам надо определить их в один из выделенных на примерых кластерах)

In [ ]:
bert_model = BertEmbeddings('bert-base-multilingual-cased')
definition_dataloader = DataLoader(ContextEmb(definitions_dataset), batch_size=8, shuffle=False)
defin_pipeline = WSIPipeline(definitions_dataset)

In [ ]:
# векторизуем с помощью берта
defin_pipeline.vectorize_contexts(definition_dataloader, bert_model)

# сохраняем, чтобы не пересчитывать
defin_pipeline.save_embeddings('data/definitions_embeddings.npy')

100%|██████████| 4/4 [00:08<00:00,  2.21s/it]


In [ ]:
# или грузим посчитанные раннее
defin_pipeline.load_embeddings('data/definitions_embeddings.npy')

In [ ]:
# грузим уже предобученные модели (потому что здесь нам надо сделать предикт в уже разбитые классы)
defin_pipeline.load_cluster_models('data/cluster_models.pkl')

# и снова получаем предсказания
defin_predicted_labels = defin_pipeline.cluster_all_dataset(fit=False)

In [ ]:
defin_predicted_labels

{'клетка': ([0, 1, 2, 3, 4], array([0, 0, 0, 0, 0], dtype=int32)),
 'фигура': ([5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
  array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1], dtype=int32)),
 'карта': ([19, 20, 21, 22, 23], array([1, 1, 1, 1, 1], dtype=int32)),
 'анализ': ([24, 25, 26], array([1, 1, 1], dtype=int32)),
 'функция': ([27, 28, 29, 30], array([1, 1, 1, 1], dtype=int32))}

In [ ]:
# теперь добавим нужный столбец в наш датасет
for w, predicted in defin_predicted_labels.items():
    definitions_dataset.loc[predicted[0], 'bert_sense'] = predicted[1]

definitions_dataset = definitions_dataset.astype({'bert_sense': 'int32'})
definitions_dataset.to_csv('data/definitions_dataset.csv', sep='\t', header=True, index=True)

In [10]:
definitions_dataset = pd.read_csv('data/definitions_dataset.csv', sep='\t', header=0, index_col=0)
definitions_dataset.head()

,word,sense_id,definition,sent_lemmatized,adagram_sense,sent,word_idx,bert_sense
0,клетка,1,Помещение для птиц и животных со стенками из ж...,помещение для птица и животное с стенка из жел...,2,Клетка - это помещение для птиц и животных со ...,0,0
1,клетка,2,"разг. Маленькая комната, тесное помещение.","разг . маленький комната , тесный помещение .",2,"Клетка - это разг. маленькая комната, тесное ...",0,0
2,клетка,3,"Способ укладки каких-л. материалов (бревен, др...","способ укладка какой - л . материал ( бревно ,...",2,Клетка - это способ укладки каких-л. материало...,0,0
3,клетка,4,"Четырехугольник, изображенный на поверхности ч...","четырёхугольник , изобразить на поверхность чт...",2,"Клетка - это четырехугольник, изображенный на ...",0,0
4,клетка,5,"Элементарная живая система, основа строения и ...","элементарный живой система , основа строение и...",4,"Клетка - это элементарная живая система, основ...",0,0


## 5 - Выберите по 2-3 значения для слова и по 5 контекстов, которые автоматически были отнесены к этим значениям. Разметьте контексты: совпадает ли значение слова в контексте со словарным значением. - 1 балл

In [11]:
# маппинг словарных значений и лейблов кластеров

def map_senses(definitions_dataset: pd.DataFrame,
               retrieved_sense_col: str,
               retrieved_sense_model: str = 'retrieved') -> Tuple[Dict[str, str], Dict[str, str]]:
    dict2retieved = {}
    retrieved2dict = {}
    for word, group in definitions_dataset.groupby(by='word'):
        for retrieved_sense, sense_group_ids in group.groupby(by=retrieved_sense_col).groups.items():
            dict_sense_ids = list(pd.unique(group.loc[sense_group_ids.values, 'sense_id']))
            retrieved2dict[f'{retrieved_sense_model}_{word}_{retrieved_sense}'] = [f'dict_{word}_{i}' for i in dict_sense_ids]

    for retrieved_sense, dict_senses in retrieved2dict.items():
        for dict_sense in dict_senses:
            dict2retieved[dict_sense] = retrieved_sense
    return retrieved2dict, dict2retieved

In [12]:
adagram2dict, dict2adagram = map_senses(definitions_dataset, 'adagram_sense', 'adagram')
bert2dict, dict2bert = map_senses(definitions_dataset, 'bert_sense', 'bert')

Я сделаю выбор значений руками, чтобы захватить все варианты, где наша модель все-таки выделила несколько значений

In [ ]:
selected_senses = [
    'dict_анализ_1',
    'dict_анализ_2',
    'dict_карта_1',
    'dict_карта_5',
    'dict_клетка_1',
    'dict_клетка_5',
    'dict_фигура_1',
    'dict_фигура_7',
    'dict_функция_2',
    'dict_функция_3',
]

А контексты буду рандомно отбирать, чтобы честно было

In [ ]:
selected_examples = defaultdict(list)
for sense in selected_senses:
    _, word, dict_sense = sense.split('_')
    bert_sense_id = int(dict2bert[sense].split('_')[-1])
    adagram_sense_id = int(dict2adagram[sense].split('_')[-1])
    word_dataset = dataset[dataset.word == word]

    bert_dataset = word_dataset[word_dataset.bert_sense == bert_sense_id]
    left = set(list(bert_dataset.index)) - set([i for _, idxs in selected_examples['bert'] for i in idxs])
    bert_selected_idx = np.random.choice(list(left), 5, replace=False)
    selected_examples['bert'].append((int(dict_sense), bert_selected_idx))

    adagram_dataset = word_dataset[word_dataset.adagram_sense == adagram_sense_id]
    left = set(list(adagram_dataset.index)) - set([i for _, idxs in selected_examples['adagram'] for i in idxs])
    adagram_selected_idx = np.random.choice(list(left), 5, replace=False)
    selected_examples['adagram'].append((int(dict_sense), adagram_selected_idx))

In [ ]:
bert_examples = []
for dict_sense, examples in selected_examples['bert']:
    bert_dataset = dataset.loc[examples, ['word', 'sent', 'bert_sense']]
    bert_dataset['sense_id'] = dict_sense
    bert_examples.append(bert_dataset)
bert_dataset = pd.concat(bert_examples)
bert_dataset.to_csv('data/bert_eval_dataset.csv', sep='\t', header=True, index=True)

In [ ]:
adagram_examples = []
for dict_sense, examples in selected_examples['adagram']:
    adagram_dataset = dataset.loc[examples, ['word', 'sent', 'adagram_sense']]
    adagram_dataset['sense_id'] = dict_sense
    adagram_examples.append(adagram_dataset)
adagram_dataset = pd.concat(adagram_examples)
adagram_dataset.to_csv('data/adagram_eval_dataset.csv', sep='\t', header=True, index=True)

Дальше я вручную разметила отобранные контексты по словарным значениям - полностью можно посмотреть в 'bert_eval_dataset_manual.tsv' и 'adagram_eval_dataset_manual.tsv'

In [ ]:
bert_dataset_manual = pd.read_csv('data/bert_eval_dataset_manual.tsv', sep='\t', header=0, index_col=0)
bert_dataset_manual = bert_dataset_manual.astype({'true': int})
bert_dataset_manual.head()

,word,sent,true,sense_id,bert_sense
814,анализ,Анализ различий между двумя группами испытуемы...,3,1,1
901,анализ,Анализ проблемы,3,1,1
764,анализ,Анализ ситуации приводит к печальному выводу :...,3,1,1
705,анализ,"Анализ всех этих систем понадобился , чтобы оц...",1,1,1
735,анализ,"Анализ всех работ , которые проводились на это...",3,1,1


In [ ]:
adagram_dataset_manual = pd.read_csv('data/adagram_eval_dataset_manual.tsv', sep='\t', header=0, index_col=0)
adagram_dataset_manual = adagram_dataset_manual.astype({'true': int})
adagram_dataset_manual.head()

,word,sent,sense_id,true,adagram_sense
827,анализ,"Расчёты , иллюстрирующие зависимость результат...",1,1,1
969,анализ,В Самарской области на его основе провели опро...,1,3,1
961,анализ,И ещё мне кажется очень важным иметь не только...,1,1,1
844,анализ,Однако структурный анализ стадии реализации по...,1,1,1
882,анализ,Рис . 4 содержит пример пользовательского инте...,1,1,1


## 6 - Оцените accuracy — в какой доле контекстов автоматически определенно значение для контекста совпадает с автоматически определенным значением толкования. - 1 балл

In [ ]:
bert_accuracy = (bert_dataset_manual.true == bert_dataset_manual.sense_id).mean()
print(f'Accuracy of WSD using BERT: {bert_accuracy}')

Accuracy of WSD using BERT: 0.28


In [ ]:
adagram_accuracy = (adagram_dataset_manual.true == adagram_dataset_manual.sense_id).mean()
print(f'Accuracy of WSD using Adagram: {adagram_accuracy}')

Accuracy of WSD using Adagram: 0.38


## 7 - Сравните результаты двух методов WSD, объясните, почему результаты совпадают или расходятся (и покажите на конкретных примерах) - максимум 2 балла

Во-первых, оба метода стабильно выделяют меньшее количесво значений, чем в словарной статье. Но мне кажется, что это нормально, учитывая, что в словарях часто выделяются слишком fine-grained значения (это очень заметно, когда начинаешь вручную размечать и понимаешь, что очень сложно определиться между такими очень специфичными значениями). Во-вторых, даже если модель поделила контексты на несколько значений, то определения часто в итоге попадают в один и тот же кластер. Например, для слова "карта" adagram разбил контексты на 5 кластеров, а bert на 4, но при этом словарные определения попали в один или два класса:

In [16]:
for dict_sense, adagram_sense in dict2adagram.items():
    if dict_sense.split('_')[1] == 'карта':
        print(dict_sense, '->', adagram_sense)

dict_карта_1 -> adagram_карта_0
dict_карта_2 -> adagram_карта_0
dict_карта_3 -> adagram_карта_0
dict_карта_4 -> adagram_карта_0
dict_карта_5 -> adagram_карта_3


In [17]:
for dict_sense, bert_sense in dict2bert.items():
    if dict_sense.split('_')[1] == 'карта':
        print(dict_sense, '->', bert_sense)

dict_карта_1 -> bert_карта_1
dict_карта_2 -> bert_карта_1
dict_карта_3 -> bert_карта_1
dict_карта_4 -> bert_карта_1
dict_карта_5 -> bert_карта_1


Я думаю, это в первую очередь связано с тем, что определения в словарных статьях как правило очень короткие и достаточно абстрактные и такого контекста недостаточно для того, чтобы адекватно оценить значение. Еще одна возможная проблема это качество самой словарной статьи - например, в словаре Ефремовой для слова "карта" нет значения для пластиковых/банковских карт. Я думаю для такой задачи лучше брать более современный словарь, который активно пополняется и редактируются (сейчас понимаю, что надо было брать викисловарь - там и статьи структурированне для парсинга и значения более актуальные)

Если смотреть на этапе WSI то модели примерно одинаково кластеризуют (обе так себе если честно, не всегда попадая). Но вот на этапе приписывания словарных значений AdaGram справляется получше (по крайней мере она не запихивает все определения в один класс). Возможно, могло бы помочь использовать определения вместе с примерами, которые часто приводятся вместе с ними в словарных статьях. Можно было бы прям взять каждое предложение из примера значения, кластеризовать, а потом значению приписать тот лейбл, куда попала бОльшая часть предложений из примера.